In [1]:
from typing import Optional, List, Union, Annotated

from datetime import date, datetime
from pydantic import BaseModel, Field

from langchain_core.messages import SystemMessage, HumanMessage

from langchain_google_vertexai import ChatVertexAI

from langgraph.graph import END, START, StateGraph
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver

from promptgit import Prompt

In [2]:
system = Prompt(
    prompt = 'You are an executive assistant to {user}. The tasks are {tasks}', variables=['user', 'tasks']
)

In [3]:
def object_to_xml(data: Union[dict, bool], root='object', ignore=None):
    if not ignore:
        ignore = []
    elif isinstance(ignore, str):
        ignore = [ignore]
    xml = f'<{root}>'
    if isinstance(data, dict):
        for key, value in data.items():
            if not(key in ignore):
                xml += object_to_xml(value, key, ignore)

    elif isinstance(data, (list, tuple, set)):
        for item in data:
            xml += object_to_xml(item, 'item', ignore)

    else:
        xml += str(data)

    xml += f'</{root}>'
    return xml

In [4]:
class Task(BaseModel):
    id: Optional[int] = None
    due: date
    description: str
    status: int = 0

class TaskList(BaseModel):
    last_update: datetime = Field(default_factory=datetime.now)
    tasks: List[Task] = Field(default = [])

    def __str__(self):
        return object_to_xml(self.model_dump()['tasks'], 'task')
    
    def add(self, due: date, description: str, status: int = 0):
        if len(self.tasks) > 0:
            id = max([t.id for t in self.tasks]) + 1
        else:
            id = 1
        self.tasks.append(Task(id = id, due = due, description = description, status = status))

In [5]:
l = TaskList()
l.add(date(2024, 10, 22), 'submit chemistry homework')
l.add(date(2024, 10, 23), 'second chapter in chemistry texbook')

In [6]:
tasks = {
    42: l
}

memory = {
    42: {
        'user': 'Boris Tvaroska',
        'date': 'Oct 22, 2024'
    }
}

In [7]:
class TaskState(BaseModel):
    user_id: int
    tasks: TaskList = None
    messages: Annotated[list, add_messages] = []

In [8]:
TaskState(user_id=42)

TaskState(user_id=42, tasks=None, messages=[])

In [9]:
llm = ChatVertexAI(model='gemini-1.5-flash-002')

def initiate(state: TaskState):
    return {'messages': [
        SystemMessage(str(system).format(tasks=str(tasks[state.user_id]) ,**memory[state.user_id])), 
        HumanMessage('What are my tasks for today/')]
           }

def chatbot(state: TaskState):
    return {"messages": [llm.invoke(state.messages)]}

In [10]:
graph_builder = StateGraph(TaskState)

graph_builder.add_node("initiate", initiate)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "initiate")
graph_builder.add_edge("initiate", "chatbot")
graph_builder.add_edge("chatbot", END)

In [11]:
checkpointer = MemorySaver()
graph = graph_builder.compile(checkpointer=checkpointer, interrupt_before=['chatbot'])

In [12]:
config = {"configurable": {"thread_id": "1"}}

In [13]:
graph.invoke({'user_id': 42}, config)

{'user_id': 42,
 'messages': [SystemMessage(content='You are an executive assistant to Boris Tvaroska. The tasks are <task><item><id>1</id><due>2024-10-22</due><description>submit chemistry homework</description><status>0</status></item><item><id>2</id><due>2024-10-23</due><description>second chapter in chemistry texbook</description><status>0</status></item></task>', additional_kwargs={}, response_metadata={}, id='a6c78a22-fb61-4943-8118-e267007ba6b0'),
  HumanMessage(content='What are my tasks for today/', additional_kwargs={}, response_metadata={}, id='e0478c95-e140-4139-a389-58ce65d8ee52')]}

In [15]:
graph.get_state(config)

StateSnapshot(values={'user_id': 42, 'messages': [SystemMessage(content='You are an executive assistant to Boris Tvaroska. The tasks are <task><item><id>1</id><due>2024-10-22</due><description>submit chemistry homework</description><status>0</status></item><item><id>2</id><due>2024-10-23</due><description>second chapter in chemistry texbook</description><status>0</status></item></task>', additional_kwargs={}, response_metadata={}, id='a6c78a22-fb61-4943-8118-e267007ba6b0'), HumanMessage(content='What are my tasks for today/', additional_kwargs={}, response_metadata={}, id='e0478c95-e140-4139-a389-58ce65d8ee52')]}, next=('chatbot',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1ef923e6-8db8-6ffc-8001-2dfde4ed284e'}}, metadata={'source': 'loop', 'writes': {'initiate': {'messages': [SystemMessage(content='You are an executive assistant to Boris Tvaroska. The tasks are <task><item><id>1</id><due>2024-10-22</due><description>submit chemistry homework</d